In [152]:
import sys
import numpy as np
import random
import copy

random.seed(10)

# mbnpy toolkit
sys.path.append(r"C:\Users\jb622s\git\BNS-JT\BNS_JT") 
from BNS_JT import cpm, variable

# Problem

## Example
<figure>
<img src="img/house_power_prob.png" style="width:400px">
</figure>

## Bayesian network
<figure>
<img src="img/house_power_bn.png" style="width:600px">
</figure>



# Complexity of the following computation

## Offline
(no. of samples) x (damage assessments) + (no. of samples) x (repair simulation)

## Online (when one house is selected for display)
min-max operation over (no. of connected substations) x (distances to substations) elements

# Offline computation and data storage

## Quantification of probability distributions (using matrix-based Bayesian network (MBN))

### Hazard node

In [153]:
cpms = {}
vars = {}

# hazard scenario
vars['h'] = variable.Variable( name='h', B=np.eye(3), values=['s1', 's2', 's3'] ) # 3 hazard scenarios
cpms['h'] = cpm.Cpm( variables=[vars['h']], no_child=1, C=np.array([0,1,2]), p=[0.5,0.2,0.3] )


### Structural performance

In [154]:
# substations
n_sub = 2
C_sub = np.array([[0,0], [0,1], [0,2], [1,0], [1,1], [1,2]])
for i in range(n_sub):
    nam = 'xs' + str(i)
    vars[nam] = variable.Variable(name=nam, B=np.eye(2), values=['f','s']) # failure or survival

    p = np.random.uniform( size=len(C_sub) ) # randomly generated probabilities just for demonstration
    p[0:3] = p[0:3] / np.sum(p[0:3])
    p[3:6] = p[3:6] / np.sum(p[3:6])

    cpms[nam] = cpm.Cpm( variables=[vars[nam], vars['h']], no_child=1, C = C_sub, p = p )

# poles
n_pol = 8
C_pol = copy.deepcopy( C_sub )
for i in range(n_pol):
    nam = 'xp' + str(i)
    vars[nam] = variable.Variable(name=nam, B=np.eye(2), values=['f','s']) 

    p = np.random.uniform( size=len(C_pol) ) 
    p[0:3] = p[0:3] / np.sum(p[0:3])
    p[3:6] = p[3:6] / np.sum(p[3:6])

    cpms[nam] = cpm.Cpm( variables=[vars[nam], vars['h']], no_child=1, C = C_pol, p = p )

### Down time (and recovery plan)

We assume a simple recovery plan that it takes 1 day to repair a failed components, and the order is sub0 -> sub1 -> pol0 -> ... -> pol8.

In [155]:
max_tim = n_sub + n_pol 

C_dum = np.empty((0,1+n_sub+n_pol), dtype='int32') # dummies: C and p are not quantified now (too much memory for exhaustive storage--this will be addressed by performing sampling.)
p_dum = []

# substations
for i in range(n_sub):
    nam = 'ts' + str(i)
    vars[nam] = variable.Variable(name=nam, B=np.eye(1+max_tim), values=list(range(1+max_tim))) # 0 days to (max_tim) days
    cpms[nam] = cpm.Cpm( variables=[vars[nam]] + [vars['xs'+str(i)] for i in range(n_sub)] + [vars['xp'+str(i)] for i in range(n_pol)], no_child=1, C = C_dum, p = p_dum ) 

# poles
for i in range(n_pol):
    nam = 'tp' + str(i)
    vars[nam] = variable.Variable(name=nam, B=np.eye(1+max_tim), values=list(range(1+max_tim)))
    cpms[nam] = cpm.Cpm( variables=[vars[nam]] + [vars['xs'+str(i)] for i in range(n_sub)] + [vars['xp'+str(i)] for i in range(n_pol)], no_child=1, C = C_dum, p = p_dum )

## Inference by MCS

### Hazard

In [156]:
n_sam = 10
cpms_s = {} # CPM of samples

# hazard 
C_hs = np.empty((n_sam,1), dtype='int32')
p_hs = np.zeros((n_sam,1)) # dummy
q_hs = np.empty((n_sam,1))
si_hs = np.empty((n_sam,1), dtype='int32')
for i in range(n_sam):
    h1 = random.choices( [s[0] for s in cpms['h'].C], weights = cpms['h'].p )
    C_hs[i], q_hs[i], si_hs[i] = h1, cpms['h'].p[h1], i
    
cpms_s['h'] = cpm.Cpm( variables=[vars['h']], no_child=1, C=C_hs, p = p_hs, q = q_hs, sample_idx=si_hs )

print(C_hs, q_hs, si_hs)


[[1]
 [0]
 [1]
 [0]
 [2]
 [2]
 [1]
 [0]
 [1]
 [0]] [[0.2]
 [0.5]
 [0.2]
 [0.5]
 [0.3]
 [0.3]
 [0.2]
 [0.5]
 [0.2]
 [0.5]] [[0]
 [1]
 [2]
 [3]
 [4]
 [5]
 [6]
 [7]
 [8]
 [9]]


### Structural performance

In [157]:
def sample_x( cpm_x, cpm_hs, n_sam, vars ):

    C_xs = np.empty((n_sam,1), dtype='int32')
    p_xs = np.zeros((n_sam,1)) # dummy
    q_xs = np.empty((n_sam,1))
    si_xs = np.empty((n_sam,1), dtype='int32')
    for i in range(n_sam):
        h1 = cpms_s['h'].C[i]
        cpm_x1 = cpm.condition( [cpm_x], [vars['h']], h1 )
        x1 = random.choices( [s[0] for s in cpm_x1[0].C], weights = cpm_x1[0].p )
        C_xs[i], q_xs[i], si_xs[i] = x1, cpm_x1[0].p[x1], i

    cpm_xs = cpm.Cpm( variables=[cpm_x.variables[0]], no_child=1, C=C_xs, p = p_xs, q = q_xs, sample_idx=si_xs )

    return cpm_xs


In [158]:
# substations 
for i in range(n_sub):
    nam = 'xs' + str(i)
    cpms_s[nam] = sample_x( cpms[nam], cpms_s['h'], n_sam, vars )

# poles
for i in range(n_pol):
    nam = 'xp' + str(i)
    cpms_s[nam] = sample_x( cpms[nam], cpms_s['h'], n_sam, vars )

### Downtime

In [159]:
rep_ord = ['xs'+str(i) for i in range(n_sub)] + ['xp'+str(i) for i in range(n_pol)] # repair order

p_ts = np.ones((n_sam,1)) 
q_ts = np.zeros((n_sam,1)) # dummy

for i, x in enumerate(rep_ord):

    C_ts = np.empty((n_sam,1), dtype='int32')
    si_ts = np.empty((n_sam,1), dtype='int32')

    for j in range(n_sam):
        x_st = cpms_s[x].C[j][0]

        if vars[x].values[x_st] == 's':
            x_dt = 0 # no downtime
        else:
            xs = rep_ord[:(i+1)]
            x_dt = 0
            for y in xs:
                y_st = cpms_s[y].C[j][0]
                if vars[y].values[y_st] == 'f':
                    x_dt += 1

        C_ts[j], si_ts[j] = x_dt, j
    
    nam = 't' + x[1::]
    cpms_s[nam] = cpm.Cpm(variables=[vars[nam]], no_child=1, C = C_ts, p=p_ts, q = q_ts, sample_idx= si_ts)

    
    

## House loss estimation - disconnection duration: by a cut-set of link-sets (i.e. min-max)

In [160]:
cut_lin = {} # cut-set of link-sets for each house

cut_lin['h0'] = [['s0', 'p0']]
cut_lin['h1'] = [['s0', 'p0', 'p1']]
cut_lin['h2'] = [['s0', 'p2']]
cut_lin['h3'] = [['s0', 'p2', 'p3'], ['s1','p4']]
cut_lin['h4'] = [['s0', 'p2', 'p3', 'p5'], ['s1', 'p4','p5']]
cut_lin['h5'] = [['s1','p6']]
cut_lin['h6'] = [['s1','p6', 'p7']]

In [165]:
def get_down_time( h_id, cut_lin, cpms_s, n_sam ):
    dts = np.empty( (n_sam,1) )
    n_lin = len(cut_lin[h_id])

    for i in range(n_sam):
        dts_i = np.empty( (n_lin,) )
        for j, lin in enumerate(cut_lin[h_id]):
            dts_lin = [cpms_s['t'+x].C[i][0] for x in lin]
            dts_i[j] = max( dts_lin )
        
        dt1 = np.amin(dts_i)
        dts[i] = dt1


    # Result
    dt_m = np.mean(dts) # mean
    dt_s = np.std(dts) # standard deviation
    dt_cov = dt_s / dt_m # Coefficient of variance (c.o.v.)

    print( '[' + h_id + '] ..' )
    print('Expected down time: ', dt_m)
    print(f'Sampling c.o.v.: {dt_cov*100: 2.1f} %' )

# Online computation: User chooses a house

In [166]:
user_h = 'h1' # user chooses a house

get_down_time( user_h, cut_lin, cpms_s, n_sam )

[h1] ..
Expected down time:  1.9
Sampling c.o.v.:  28.3 %


In [167]:
# all results

for h in cut_lin:
    get_down_time( h, cut_lin, cpms_s, n_sam )

[h0] ..
Expected down time:  1.3
Sampling c.o.v.:  69.2 %
[h1] ..
Expected down time:  1.9
Sampling c.o.v.:  28.3 %
[h2] ..
Expected down time:  1.2
Sampling c.o.v.:  97.2 %
[h3] ..
Expected down time:  1.5
Sampling c.o.v.:  112.5 %
[h4] ..
Expected down time:  2.5
Sampling c.o.v.:  76.4 %
[h5] ..
Expected down time:  2.2
Sampling c.o.v.:  97.1 %
[h6] ..
Expected down time:  4.2
Sampling c.o.v.:  48.6 %


NB: Coefficients of variance (c.o.v.) can be reduced by increasing the number of samples.